In [ ]:
import cv2
import urllib.request
import requests
import numpy as np
# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://192.168.0.179:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                requests.get('http://192.168.0.179/signal?300')
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()
 